# Day12_1: 데이터 전처리 for ML - 정답 노트북

이 노트북은 Day12_1 실습 퀴즈의 정답과 상세 해설을 포함합니다.

---

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, RobustScaler,
    LabelEncoder, OneHotEncoder, PolynomialFeatures
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

# Titanic 데이터 로드
try:
    import seaborn as sns
    titanic = sns.load_dataset('titanic')
except:
    url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
    titanic = pd.read_csv(url)
    titanic.columns = titanic.columns.str.lower()

print("환경 설정 완료!")

---

## Q1. 데이터 분할 ⭐

### 문제
다음 데이터를 70% 훈련, 30% 테스트로 분할하세요.

```python
X = np.arange(100).reshape(50, 2)
y = np.random.randint(0, 2, 50)
```

**요구사항**: random_state=42, stratify 적용

In [ ]:
# 정답 코드
np.random.seed(42)  # 재현성을 위해 고정
X = np.arange(100).reshape(50, 2)
y = np.random.randint(0, 2, 50)

# train_test_split으로 70:30 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,       # 30% 테스트
    random_state=42,     # 재현성
    stratify=y           # 클래스 비율 유지
)

# 결과 확인
print(f"전체 데이터: {len(X)}개")
print(f"훈련 데이터: {len(X_train)}개 ({len(X_train)/len(X)*100:.0f}%)")
print(f"테스트 데이터: {len(X_test)}개 ({len(X_test)/len(X)*100:.0f}%)")
print(f"\n클래스 비율 확인:")
print(f"  전체: {y.mean():.3f}")
print(f"  훈련: {y_train.mean():.3f}")
print(f"  테스트: {y_test.mean():.3f}")

In [ ]:
# 테스트
assert len(X_train) == 35, "훈련 데이터는 35개여야 합니다"
assert len(X_test) == 15, "테스트 데이터는 15개여야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- `train_test_split()`은 sklearn의 데이터 분할 함수입니다
- `test_size=0.3`은 전체의 30%를 테스트로 사용한다는 의미

**핵심 개념**:
- `random_state`: 동일한 분할 결과를 재현하기 위한 시드값
- `stratify`: 타겟 변수의 클래스 비율을 유지하며 분할

**대안**:
- `train_size=0.7`로도 같은 결과
- `shuffle=True`가 기본값 (데이터 섞기)

**흔한 실수**:
- stratify에 X를 넣는 실수 (y를 넣어야 함)
- random_state를 생략하면 매번 다른 결과

**실무 팁**:
- 불균형 데이터에서는 반드시 stratify 사용
- 일반적으로 70:30 또는 80:20 분할

---

## Q2. StandardScaler 적용 ⭐

### 문제
다음 데이터에 StandardScaler를 적용하고, 변환 후 평균과 표준편차를 확인하세요.

```python
data = np.array([[10, 1000], [20, 2000], [30, 3000], [40, 4000]])
```

In [ ]:
# 정답 코드
data = np.array([[10, 1000], [20, 2000], [30, 3000], [40, 4000]])

# StandardScaler 생성 및 적용
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# 결과 확인
print("원본 데이터:")
print(data)
print(f"\n원본 평균: {data.mean(axis=0)}")
print(f"원본 표준편차: {data.std(axis=0)}")

print("\n변환된 데이터:")
print(data_scaled.round(3))
print(f"\n변환 후 평균: {data_scaled.mean(axis=0).round(10)}")
print(f"변환 후 표준편차: {data_scaled.std(axis=0).round(3)}")

In [ ]:
# 테스트
assert np.allclose(data_scaled.mean(axis=0), [0, 0]), "평균이 0이어야 합니다"
assert np.allclose(data_scaled.std(axis=0), [1, 1]), "표준편차가 1이어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- StandardScaler는 z-score 표준화를 수행
- 각 특성에서 평균을 빼고 표준편차로 나눔

**핵심 개념**:
- 공식: z = (x - mean) / std
- 결과: 평균 0, 표준편차 1

**대안**:
- 수동 계산: `(data - data.mean(axis=0)) / data.std(axis=0)`

**흔한 실수**:
- fit과 transform을 분리하지 않음 (테스트 데이터 처리 시 문제)
- 전체 데이터로 fit하고 훈련/테스트 모두 transform (Data Leakage)

**실무 팁**:
- 로지스틱 회귀, SVM 등에 필수
- 트리 기반 모델은 스케일링 불필요

---

## Q3. MinMaxScaler 범위 지정 ⭐⭐

### 문제
MinMaxScaler를 사용하여 데이터를 -1 ~ 1 범위로 변환하세요.

```python
values = np.array([[0], [25], [50], [75], [100]])
```

In [ ]:
# 정답 코드
values = np.array([[0], [25], [50], [75], [100]])

# MinMaxScaler로 -1 ~ 1 범위 지정
scaler = MinMaxScaler(feature_range=(-1, 1))
values_scaled = scaler.fit_transform(values)

# 결과 확인
print("원본 -> 변환:")
for orig, scaled in zip(values.flatten(), values_scaled.flatten()):
    print(f"  {orig:3d} -> {scaled:6.2f}")

print(f"\n최솟값: {values_scaled.min():.2f}")
print(f"최댓값: {values_scaled.max():.2f}")

In [ ]:
# 테스트
assert values_scaled.min() == -1, "최솟값이 -1이어야 합니다"
assert values_scaled.max() == 1, "최댓값이 1이어야 합니다"
assert values_scaled[2][0] == 0, "중간값(50)이 0이어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- MinMaxScaler의 `feature_range` 파라미터로 변환 범위 지정
- 기본값은 (0, 1)

**핵심 개념**:
- 공식: x_scaled = (x - min) / (max - min) * (max_range - min_range) + min_range

**대안**:
- 수동 계산 가능하나 코드가 복잡해짐

**흔한 실수**:
- feature_range를 리스트로 전달 (튜플이어야 함)

**실무 팁**:
- 신경망에서 tanh 활성화 함수 사용 시 -1~1 범위 유용
- 이미지 데이터는 보통 0~1 범위 사용

---

## Q4. LabelEncoder 적용 ⭐⭐

### 문제
다음 등급 데이터를 숫자로 인코딩하고, 역변환도 해보세요.

```python
grades = ['Bronze', 'Silver', 'Gold', 'Silver', 'Bronze', 'Gold']
```

In [ ]:
# 정답 코드
grades = ['Bronze', 'Silver', 'Gold', 'Silver', 'Bronze', 'Gold']

# LabelEncoder 생성 및 적용
le = LabelEncoder()
grades_encoded = le.fit_transform(grades)

print("클래스 목록:", le.classes_)
print("\n인코딩 결과:")
for orig, enc in zip(grades, grades_encoded):
    print(f"  {orig} -> {enc}")

# 역변환
decoded = le.inverse_transform([0, 1, 2])
print(f"\n역변환 [0,1,2] -> {decoded}")

In [ ]:
# 테스트
assert len(le.classes_) == 3, "3개의 클래스가 있어야 합니다"
assert grades_encoded[0] == 0, "Bronze는 0이어야 합니다"
assert le.inverse_transform([1])[0] == 'Gold', "1은 Gold로 역변환되어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- LabelEncoder는 문자열을 알파벳 순서로 0부터 정수 매핑
- Bronze(0) < Gold(1) < Silver(2)

**핵심 개념**:
- 클래스가 알파벳 순으로 정렬됨
- `classes_` 속성으로 매핑 확인 가능

**대안**:
- OrdinalEncoder: 순서를 직접 지정 가능
- 딕셔너리 매핑: `{'Bronze': 0, 'Silver': 1, 'Gold': 2}`

**흔한 실수**:
- 순서가 중요한 경우 알파벳 순 인코딩이 의미 없음
- 새로운 카테고리가 등장하면 에러 발생

**실무 팁**:
- 타겟 변수나 이진 범주에 주로 사용
- 순서가 있는 범주는 OrdinalEncoder 권장

---

## Q5. OneHotEncoder 적용 ⭐⭐

### 문제
다음 색상 데이터를 원-핫 인코딩하세요.

```python
colors = np.array([['Red'], ['Blue'], ['Green'], ['Blue'], ['Red']])
```

In [ ]:
# 정답 코드
colors = np.array([['Red'], ['Blue'], ['Green'], ['Blue'], ['Red']])

# OneHotEncoder 적용
ohe = OneHotEncoder(sparse_output=False)
colors_encoded = ohe.fit_transform(colors)

print("카테고리:", ohe.categories_)
print("특성 이름:", ohe.get_feature_names_out(['color']))
print("\n원-핫 인코딩 결과:")
result_df = pd.DataFrame(
    colors_encoded,
    columns=ohe.get_feature_names_out(['color'])
)
result_df['원본'] = colors.flatten()
print(result_df)

In [ ]:
# 테스트
assert colors_encoded.shape == (5, 3), "결과 shape은 (5, 3)이어야 합니다"
assert colors_encoded.sum() == 5, "각 행에 1이 하나씩 있어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- OneHotEncoder는 각 카테고리를 별도의 이진 컬럼으로 변환
- sparse_output=False로 밀집 배열 반환

**핵심 개념**:
- 3개 카테고리 -> 3개 컬럼
- 해당 카테고리만 1, 나머지 0

**대안**:
- pandas: `pd.get_dummies(colors.flatten())`
- drop='first'로 다중공선성 방지 가능

**흔한 실수**:
- 2D 배열이 아닌 1D 배열 입력
- sparse_output=True일 때 dense 연산 시도

**실무 팁**:
- 선형 모델에서는 drop='first' 권장
- 트리 모델은 원-핫보다 타겟 인코딩이 효과적일 수 있음

---

## Q6. PolynomialFeatures 생성 ⭐⭐⭐

### 문제
2개의 특성에서 2차 다항 특성을 생성하고, 특성 이름을 출력하세요.

```python
X = np.array([[1, 2], [3, 4], [5, 6]])
```

**요구사항**: bias 항 제외

In [ ]:
# 정답 코드
X = np.array([[1, 2], [3, 4], [5, 6]])

# 2차 다항 특성 생성 (bias 제외)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

# 특성 이름
feature_names = poly.get_feature_names_out(['x1', 'x2'])
print("특성 이름:", feature_names)
print(f"특성 수: {len(feature_names)}")

# 결과 확인
print("\n다항 특성 결과:")
result_df = pd.DataFrame(X_poly, columns=feature_names)
print(result_df)

In [ ]:
# 테스트
assert X_poly.shape == (3, 5), "결과 shape은 (3, 5)여야 합니다: x1, x2, x1^2, x1*x2, x2^2"
assert 'x1 x2' in feature_names, "상호작용 특성 x1*x2가 있어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- degree=2: 최대 2차 다항식
- include_bias=False: 상수항(1) 제외

**핵심 개념**:
- 2개 특성, degree=2일 때: x1, x2, x1^2, x1*x2, x2^2 (5개)
- 상호작용 특성(x1*x2)으로 비선형 관계 포착

**대안**:
- interaction_only=True: 상호작용만 (x1, x2, x1*x2)

**흔한 실수**:
- bias 포함 시 상수항 1이 추가됨 (대부분 불필요)
- 특성 수가 급격히 증가 (차원의 저주)

**실무 팁**:
- 다항 회귀에서 비선형 관계 모델링
- 특성이 많으면 degree=2도 폭발적 증가 주의

---

## Q7. 수치형 Pipeline 구축 ⭐⭐⭐

### 문제
결측치 대치(중앙값) -> 표준화 파이프라인을 만들고 적용하세요.

```python
data_with_nan = np.array([[1, 10], [2, np.nan], [np.nan, 30], [4, 40]])
```

In [ ]:
# 정답 코드
data_with_nan = np.array([[1, 10], [2, np.nan], [np.nan, 30], [4, 40]])

# 파이프라인 구축
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # 중앙값 대치
    ('scaler', StandardScaler())                     # 표준화
])

# 적용
data_transformed = numeric_pipeline.fit_transform(data_with_nan)

print("원본 데이터 (결측치 포함):")
print(data_with_nan)

print("\n변환된 데이터:")
print(data_transformed.round(3))

print(f"\n변환 후 평균: {data_transformed.mean(axis=0).round(10)}")
print(f"변환 후 표준편차: {data_transformed.std(axis=0).round(3)}")

In [ ]:
# 테스트
assert not np.isnan(data_transformed).any(), "결측치가 없어야 합니다"
assert np.allclose(data_transformed.mean(axis=0), [0, 0], atol=1e-10), "평균이 0이어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- Pipeline은 여러 변환기를 순서대로 연결
- (이름, 변환기) 튜플의 리스트로 정의

**핵심 개념**:
- 순차 실행: imputer -> scaler
- fit_transform()으로 한 번에 적용

**대안**:
- make_pipeline(): 이름 자동 생성

**흔한 실수**:
- 순서 잘못 (스케일링 후 결측치 대치 불가)
- 테스트 데이터에 fit_transform 사용

**실무 팁**:
- 파이프라인은 코드 재사용과 실수 방지에 효과적
- 교차 검증 시 자동으로 각 Fold마다 fit/transform 분리

---

## Q8. K-Fold 교차 검증 ⭐⭐⭐⭐

### 문제
Titanic 데이터로 5-Fold 교차 검증을 수행하고, 각 Fold의 정확도를 출력하세요.

**요구사항**:
- age, fare 특성 사용 (결측치 제거)
- LogisticRegression 모델
- StratifiedKFold 사용

In [ ]:
# 정답 코드
# 데이터 준비
X_cv = titanic[['age', 'fare']].dropna()
y_cv = titanic.loc[X_cv.index, 'survived']

# 스케일링 적용
scaler = StandardScaler()
X_cv_scaled = scaler.fit_transform(X_cv)

# 모델 생성
model = LogisticRegression(max_iter=1000)

# 방법 1: cross_val_score 사용
cv_scores = cross_val_score(model, X_cv_scaled, y_cv, cv=5, scoring='accuracy')

print("cross_val_score 결과:")
for i, score in enumerate(cv_scores, 1):
    print(f"  Fold {i}: {score:.4f}")
print(f"\n평균: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")

In [ ]:
# 방법 2: StratifiedKFold 직접 사용
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = []
print("\nStratifiedKFold 직접 구현:")
for fold, (train_idx, val_idx) in enumerate(skfold.split(X_cv_scaled, y_cv), 1):
    X_train_fold = X_cv_scaled[train_idx]
    X_val_fold = X_cv_scaled[val_idx]
    y_train_fold = y_cv.iloc[train_idx]
    y_val_fold = y_cv.iloc[val_idx]
    
    model.fit(X_train_fold, y_train_fold)
    score = model.score(X_val_fold, y_val_fold)
    scores.append(score)
    print(f"  Fold {fold}: {score:.4f}")

print(f"\n평균: {np.mean(scores):.4f}")

In [ ]:
# 테스트
assert len(cv_scores) == 5, "5개 Fold 결과가 있어야 합니다"
assert all(0 <= s <= 1 for s in cv_scores), "정확도는 0~1 사이여야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- cross_val_score: 간편한 교차 검증
- StratifiedKFold: 클래스 비율 유지하며 분할

**핵심 개념**:
- 5-Fold: 데이터를 5등분, 1개씩 검증용으로 순환
- 평균 +/- 2*std로 신뢰 구간 표현

**대안**:
- KFold: 클래스 비율 무시 (균형 데이터용)
- RepeatedStratifiedKFold: 여러 번 반복

**흔한 실수**:
- 전체 데이터로 스케일링 후 교차 검증 (Data Leakage)
- shuffle=False일 때 순차 데이터 문제

**실무 팁**:
- 불균형 데이터는 반드시 StratifiedKFold
- 데이터가 적으면 10-Fold 권장

---

## Q9. ColumnTransformer 구축 ⭐⭐⭐⭐

### 문제
다음 데이터프레임에 대해 수치형/범주형을 동시에 처리하는 ColumnTransformer를 만드세요.

```python
df = pd.DataFrame({
    'age': [25, 30, np.nan, 40],
    'salary': [50000, 60000, 70000, np.nan],
    'department': ['IT', 'HR', 'IT', 'Sales'],
    'gender': ['M', 'F', 'M', 'F']
})
```

**요구사항**:
- 수치형: 중앙값 대치 + 표준화
- 범주형: 최빈값 대치 + 원-핫 인코딩

In [ ]:
# 정답 코드
df = pd.DataFrame({
    'age': [25, 30, np.nan, 40],
    'salary': [50000, 60000, 70000, np.nan],
    'department': ['IT', 'HR', 'IT', 'Sales'],
    'gender': ['M', 'F', 'M', 'F']
})

# 컬럼 정의
numeric_cols = ['age', 'salary']
categorical_cols = ['department', 'gender']

# 수치형 파이프라인
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 범주형 파이프라인
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

# ColumnTransformer 구성
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_cols),
        ('cat', categorical_pipeline, categorical_cols)
    ]
)

# 적용
df_transformed = preprocessor.fit_transform(df)

print("원본 데이터:")
print(df)

print(f"\n변환 후 shape: {df_transformed.shape}")
print("\n변환된 특성 이름:")
feature_names = preprocessor.get_feature_names_out()
for i, name in enumerate(feature_names):
    print(f"  {i}: {name}")

print("\n변환된 데이터:")
print(pd.DataFrame(df_transformed, columns=feature_names).round(3))

In [ ]:
# 테스트
assert df_transformed.shape[0] == 4, "행 수는 4개여야 합니다"
assert not np.isnan(df_transformed).any(), "결측치가 없어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- ColumnTransformer로 서로 다른 파이프라인을 컬럼별로 적용
- transformers: (이름, 파이프라인, 컬럼 리스트) 튜플

**핵심 개념**:
- 수치형과 범주형을 동시에 처리
- 각 파이프라인이 독립적으로 fit/transform

**대안**:
- remainder='passthrough': 지정 안 한 컬럼 유지
- remainder='drop': 지정 안 한 컬럼 제거 (기본값)

**흔한 실수**:
- 컬럼 이름이 인덱스로 변경됨 (get_feature_names_out으로 복구)
- 컬럼 순서가 원본과 다를 수 있음

**실무 팁**:
- 실제 프로젝트에서 거의 필수로 사용
- 모델과 함께 Pipeline으로 묶으면 완전한 ML 파이프라인

---

## Q10. 종합: Titanic ML 파이프라인 ⭐⭐⭐⭐⭐

### 문제
Titanic 데이터로 완전한 ML 파이프라인을 구축하고, 테스트 정확도를 출력하세요.

**요구사항**:
1. 사용 특성: age, fare, pclass, sex, embarked
2. 수치형(age, fare): 중앙값 대치 + StandardScaler
3. 범주형(pclass, sex, embarked): 최빈값 대치 + OneHotEncoder
4. 모델: LogisticRegression
5. 데이터 분할: 80/20, stratify, random_state=42
6. 5-Fold 교차 검증 점수도 출력

In [ ]:
# 정답 코드

# 1. 특성 정의
numeric_features = ['age', 'fare']
categorical_features = ['pclass', 'sex', 'embarked']
all_features = numeric_features + categorical_features

# 2. 수치형 파이프라인
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 3. 범주형 파이프라인
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# 4. ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# 5. 전체 파이프라인 (전처리 + 모델)
full_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# 6. 데이터 준비
X = titanic[all_features]
y = titanic['survived']

# 결측치 있는 타겟 제거
mask = y.notna()
X = X[mask]
y = y[mask]

print(f"전체 데이터: {len(X)}개")
print(f"생존율: {y.mean():.3f}")

In [ ]:
# 7. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"훈련 데이터: {len(X_train)}개")
print(f"테스트 데이터: {len(X_test)}개")

# 8. 학습 및 평가
full_pipeline.fit(X_train, y_train)

train_score = full_pipeline.score(X_train, y_train)
test_score = full_pipeline.score(X_test, y_test)

print(f"\n훈련 정확도: {train_score:.4f}")
print(f"테스트 정확도: {test_score:.4f}")

In [ ]:
# 9. 5-Fold 교차 검증
cv_scores = cross_val_score(full_pipeline, X, y, cv=5, scoring='accuracy')

print("\n5-Fold 교차 검증 결과:")
for i, score in enumerate(cv_scores, 1):
    print(f"  Fold {i}: {score:.4f}")
print(f"\n평균: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")

In [ ]:
# 10. 결과 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['모델 성능', '교차 검증 결과'])

# 훈련/테스트 정확도
fig.add_trace(
    go.Bar(x=['훈련', '테스트'], y=[train_score, test_score],
           marker_color=['steelblue', 'coral'],
           text=[f'{train_score:.3f}', f'{test_score:.3f}'],
           textposition='outside'),
    row=1, col=1
)

# 교차 검증 결과
fig.add_trace(
    go.Bar(x=[f'Fold {i}' for i in range(1, 6)], y=cv_scores,
           marker_color='mediumseagreen',
           text=[f'{s:.3f}' for s in cv_scores],
           textposition='outside'),
    row=1, col=2
)

fig.update_layout(title='Titanic ML 파이프라인 성능', showlegend=False, height=400)
fig.update_yaxes(range=[0.6, 0.9])
fig.show()

In [ ]:
# 테스트
assert test_score > 0.7, "테스트 정확도가 70% 이상이어야 합니다"
assert len(cv_scores) == 5, "5개 Fold 결과가 있어야 합니다"
assert cv_scores.mean() > 0.7, "CV 평균이 70% 이상이어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 수치형/범주형 컬럼 분리
2. 각각에 맞는 파이프라인 구축
3. ColumnTransformer로 통합
4. 모델과 함께 전체 파이프라인 구성
5. 훈련/테스트 분할 및 평가
6. 교차 검증으로 신뢰성 확인

**핵심 개념**:
- 전처리와 모델을 하나의 객체로 관리
- Data Leakage 자동 방지 (교차 검증 시 각 Fold마다 fit/transform 분리)

**대안**:
- 다른 모델 (RandomForest, XGBoost 등)
- GridSearchCV로 하이퍼파라미터 튜닝

**흔한 실수**:
- 전체 데이터로 전처리 후 분할 (Data Leakage)
- pclass를 수치형으로 처리 (순서 없는 범주형)

**실무 팁**:
- 이 패턴이 실제 ML 프로젝트의 기본 골격
- joblib.dump()로 파이프라인 저장, 배포 시 그대로 사용

---

## 📊 전체 요약

### 핵심 개념 정리

| 단계 | 도구 | 핵심 포인트 |
|------|------|------------|
| 데이터 분할 | train_test_split | stratify로 클래스 비율 유지 |
| 교차 검증 | StratifiedKFold | 여러 번 평가로 신뢰성 확보 |
| 스케일링 | StandardScaler, MinMaxScaler | 훈련 데이터로만 fit |
| 인코딩 | LabelEncoder, OneHotEncoder | 범주형을 숫자로 변환 |
| 파이프라인 | Pipeline, ColumnTransformer | 전처리 자동화 |

### 실무 체크리스트

- [ ] Data Leakage 방지: 테스트 데이터로 fit 금지
- [ ] 불균형 데이터: stratify, StratifiedKFold 사용
- [ ] 결측치 처리: SimpleImputer 활용
- [ ] 스케일링: 모델 특성에 맞게 선택
- [ ] 파이프라인: 전처리+모델 통합 관리